##Conexión de Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Sección 1 Origen de los datos

## **Introducción**

Como QFB, trabajan con muestras complejas (Datos Crudos):

*   🩸 Sangre/plasma (biomarcadores)
*   🔬 Tejidos tumorales/inflamados (biopsias)
*   🧫 Cultivos celulares (hepatocitos, líneas cancerosas)
*   🦠 Aislamientos microbianos (resistencia antimicrobiana)

**Reto:** Extraer información funcional de miles de genes en estas muestras.

**Solución:** Bioinformática para transformar datos crudos en mecanismos farmacológicos o diagnósticos.

# Beneficios:


1.   Dejar atrás los análisis unigénicos (qPCR) para adoptar enfoques multiómicos.

2.   Vincular sus muestras a mecanismos moleculares con impacto en terapias y diagnósticos


# Sección 2 Bioinformática



<fig>
<img    src="Edits/Curso.png"
        width=800
        height=800>
<figcaption>Fig. 1: Graphical abstract de la metodología.
</figure>

## Procesamiento de datos scRNA-seq

In [ ]:
# Librerias

import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np

In [ ]:
singlecell=pd.read_csv("/content/drive/MyDrive/Curso de Marzo/SingleCell.csv", header=None) #Single Cell con 3,500 muestras celulares de 15 tipos celulares
singlecell=singlecell.transpose()
singlecell

##Procesamiento de datos RNA-seq

In [ ]:
# Carga de datos

rnaseq=pd.read_csv("/content/drive/MyDrive/Curso de Marzo/RNA-seq.csv") # Dirección en la que se encuentra el archivo
print("Base de datos \n")
rnaseq = rnaseq.rename({'gene_symbol': 'Brain Samples', 'b': 'Y'}, axis=1)
rnaseq.head()


In [ ]:
ages = ["11 yrs", "40 yrs"] #Edades en las que quieres inferir la proporción celular


# Filtara el DF
filtered_df = rnaseq[rnaseq["Age"].isin(ages)]

# Eliminar columna de edad
filtered_df=filtered_df.drop("Age", axis=1)

# Mostrar los resultados
filtered_df=filtered_df.set_index(["Brain Samples"])
print("Base de datos con Indice\n")

filtered_df.head()

# Define the Google Drive path
save_path = "/content/drive/MyDrive/Curso de Marzo/"

#Exportar los datos a un archivo tipo csv
filtered_df.to_csv(save_path+"filtered_df.csv")

In [ ]:
# Carga de datos previamente segmentados
rnaseq=pd.read_csv(save_path+"filtered_df.csv", header=None)
print("Base de datos \n")
rnaseq=rnaseq.transpose()
# Save the file
np.savetxt(save_path+"Brain_Bulk.txt", rnaseq, fmt='%s', delimiter='\t')

In [ ]:
rnaseq=pd.read_csv(save_path+"Brain_Bulk.txt", header=None, delimiter="\t", index_col=0)
rnaseq

#CIBERSORTx

Link a CIBERSORTx: https://cibersortx.stanford.edu/

Pasos a seguir


1.   Crear cuenta
2.   Iniciar sesíon


# Sección 2: Origen de los datos

Este es el origen de los datos blah blah blah

In [ ]:
# Aqui va algo de texto
def imprimir_valor(valor):
    print(valor)

In [ ]:
imprimir_valor(4)

##hdWGCNA

#Sección 3 Ciencia de datos